# 2개의 층을 갖는 심층 신경망 만들기

일단 데이터를 가져와서 분리하는 것은 07-1과 같다.

In [1]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

4431872/4422102 [==============================] - 0s 0us/step


이제 입력층과 출력층 사이에 은닉층을 하나 추가해보자.

모든 은닉층은 활성화 함수를 지닌다.
- 출력층의 활성화 함수는 종류가 제한되어 있다. (시그모이드, 소프트맥스)
- 은닉층의 활성화 함수는 비교적 자유롭다. (시그모이드, 렐루 등)

은닉층에 활성화 함수를 적용하는 이유
- 두 개의 선형 방정식(뉴런)을 하나로 합치면, 중간 매개변수의 의미가 사라진다.
- $ 4a + 2 = b $, $ 3b - 5 = c $ ----> $ 12a + 1 = c $
- 은닉층에서 선형적인 산술 계산만 수행한다면 수행 역할이 없는 셈
    - 따라서 선형 계산을 적당하게 비선형적으로 비틀어 주어야 한다.
    - 그래서 활성화 함수를 적용하고 그래야 나름의 역할을 할 수 있다.
- 많이 적용하는 활성화 함수는 시그모이드

이렇게 시그모이드를 거치는 은닉층과, 소프트맥스를 거치는 출력층을 Dense 클래스로 만들어보자.
- 첫 층은 반드시 input_shape로 입력 크기를 지정해주어야 한다.

In [2]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784, ))
dense2 = keras.layers.Dense(10, activation='softmax')

은닉층에서 몇개의 뉴런을 사용할지는 특별한 기준이 없다. 이를 판단하기 위해서는 상당한 경험이 필요하다.

단, 적어도 출력층의 뉴런보다는 많아야 한다. 클래스 10개에 대한 확률을 예측해야 하는데, 은닉층에 10개보다 적은 뉴런이 있다면 부족한 정보가 전달될 것.

이제 Sequential 클래스에 위 Dense 객체들을 추가하여 심층 신경망을 만들자.

In [3]:
model = keras.Sequential([dense1, dense2]) # 여러 모델을 리스트로 묶어 전달

케라스는 모델의 summary() 함수를 통해 층에 대한 유용한 정보를 얻을 수 있다.

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


레이어의 이름은 층을 만들 때 name 변수로 지정할 수 있다.

출력 크기가 (Npne, 10) 으로
- 첫 번째 차원은 샘플의 개수. 왜 None 일까?
    - 샘플 개수가 아직 정의되지 않았기 때문
    - fit() 는 훈련 데이터를 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행한다. (미니배치 경사 하강법)
        - 이 값은 fit() 에서 batch_size 변수로 바꿀 수 있다.
    - 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정한다.
    - 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 차원을 배치 차원이라고 한다.
- 두 번째 차원은 뉴런(출력) 개수

모델 파라미터 개수
- 첫째 dense는 100개의 출력에 대한 784개의 입력 + bias = (784 + 1) * 100 = 78500
- 둘째 dense는 10개의 출력에 대한 100개의 입력 + bias = (100 + 1) * 10 = 1010

Non-trainable params: 경사 하강법으로 훈련되지 않는 파라미터의 개수

In [5]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5623 - accuracy: 0.8073
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4087 - accuracy: 0.8533
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3729 - accuracy: 0.8658
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3505 - accuracy: 0.8718
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3334 - accuracy: 0.8791


추가된 층이 성능을 약간 향상시켰다.

## 층을 추가하는 다른 방법

1. Sequential 생성자 호출과 동시에 Dense 객체를 정의하여 전달하기

In [6]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784, ), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


2. Sequential 클래스의 add() 함수를 호출하기

In [7]:
model = keras.Sequential(name = '패션 MNIST 모델')
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784, ), name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


# 렐루 함수

시그모이드의 단점
- 입력값의 크기가 조금만 커져도 1이나 0에 가까워지기 때문에,
- 은닉층이 많아질수록 극단적으로 0이나 1에 가까워지기 쉽다.

렐루(ReLU)
- $ ReLU(z) = max(0, z)$
- 입력이 양수이면 그 양수를 그대로 출력하고, 음수이면 0으로 만든다.
- 렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다.

또한, 앞선 예제에서는 28x28 이미지를 784 크기의 1차원으로 직접 펼친 후 Dense의 입력층으로 넣었지만,

케라스의 Flatten 클래스를 이용하면, 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼쳐준다.

얘도 모델에 마치 층처럼 들어가지만, 학습을 위해 기여하는 건 없다.

In [8]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28))) # 첫 층이니까 input_shape 지정
model.add(keras.layers.Dense(100, activation='relu')) # 활성화 함수로 렐루
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


flatten 층의 출력 형태를 보면, 입력 데이터의 차원을 짐작할 수 있는 것도 장점.

이제 훈련 데이터를 다시 준비해서 모델을 훈련해보자.

In [9]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
# train_scaled = train_scaled.reshape(-1, 28*28) # flatten이 있으니 reshape는 생략한다.
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [10]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5356 - accuracy: 0.8106
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3931 - accuracy: 0.8591
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3559 - accuracy: 0.8717
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3352 - accuracy: 0.8796
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3170 - accuracy: 0.8865


정말 소소하게 성능이 올랐다. 이제 검증 데이터로 평가를 해보자

In [11]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3659 - accuracy: 0.8781


[0.36592772603034973, 0.878083348274231]

# 옵티마이저

신경망 모델 구성에는 정말 다양한 하이퍼파라미터가 필요하다.
- 추가할 은닉층의 개수 및 층의 종류
- 뉴런 개수
- 활성화 함수
- 배치 사이즈
- 에포크
- 옵티마이저(경사 하강법 알고리즘) 및 그의 학습률

여기서는 여러가지 옵티마이저를 테스트해보자.

SGD: 확률적 경사 하강법. 가장 기본적인 옵티마이저.
- 이름은 SGD 이지만, 샘플을 하나씩 뽑지 않고, 기본적으로 미니배치를 사용한다.
- compile() 함수의 optimizer 매개변수를 'sgd'로 지정한다.


In [12]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

# 위 코드는 다음 코드와 정확히 일치한다.
#
#    sgd = keras.optimizers.SGD(learning_rate=0.1) # 학습률을 변경하고 싶다면
#    model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')
#
# 원래는 위와 같이 SGD 객체를 만들어야 하지만,
# 번거로움을 피하고자 optimizer='sgd'를 바로 넣을 수 있게 만들어져 있다.

SGD 외의 자주 사용하는 옵티마이저는 다음과 같다.

기본 경사 하강법 옵티마이저
- SGD 클래스의 momentum 매개변수의 기본값으 0
- 모멘텀 최적화(momentum optimization): 이전의 그래디언트를 마치 가속도처럼 사용하는 것
- Momentum: SGD에 모멘텀을 적용
    - SGD 클래스의 momentum 매개변수를 보통 0.9 이상으로 지정한다.
- Nesterov momentum: 모멘텀 최적화를 2번 반복하여 구현한다.
    - SGD 클래스의 nesterov 매개변수를 True로 지정

적응적 학습률 옵티마이저
- 모델이 최적점에 가까이 갈수록 학습률을 낮추는 것 --> 안정적으로 최적점에 수렴할 수 있음
    - 이러한 학습률을 적응적 학습률(adaptive learning rate)이라고 한다.
    - 학습률 매개변수를 튜닝하는 수고를 덜 수 있는 것이 장점.
- RMSprop과 Adagrad가 있으며 compile() 메소드의 optimizer 매개변수에서 지정 가능하다.
    - RMSprop: optimizer 매개변수의 기본값 'rmsprop'
- Adam: 모멘텀 최적화와 RMSprop의 장점을 접목한 것
    - Adam 클래스도 keras.optimizers 패키지 아래에 있다.
- 위 세 클래스는 모두 learning_rate 기본값으로 0.001을 사용한다.

In [13]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True) # Nesterov momentum 옵티마이저 객체

adagrad = keras.optimizers.Adagrad() # Adargad 옵티마이저 객체
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

rmsprop = keras.optimizers.RMSprop() # RMSprop 옵티마이저 객체
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

여기에서는 Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST 모델을 훈련해보자.
(모델 생성부터)

In [15]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

compile() 함수의 optimizer를 'adam'으로 설정하고 5번의 에포크 동안 훈련하자.

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5258 - accuracy: 0.8183
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3915 - accuracy: 0.8595
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3512 - accuracy: 0.8739
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3249 - accuracy: 0.8810
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3058 - accuracy: 0.8890


그냥 렐루를 사용한 RMSprop보다 소소하게 성능이 올라갔다.

검증 세트에 대해서도 평가해보자.

In [17]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3466 - accuracy: 0.8748


[0.3465598523616791, 0.874833345413208]

책에선 RMSprop보다 조금 높은 성능이었는데, 여기선 더 낮다. 왜그런지는 몰?루